In [85]:
import pandas as pd
from konlpy.tag import Hannanum
import yfinance as yf
import datetime
import numpy as np
import warnings
import re
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
warnings.filterwarnings('ignore')

In [65]:
def df_process (df):
    
    df['일자'] = pd.to_datetime(df['일자'], format='%Y%m%d')
    df = df.set_index('일자')

    df = df.sort_index(ascending=True) 

    df = df[['제목']].copy()

    start_date_str = str(df.index.unique()[0])[0:10]
    end_date_str = str(df.index.unique()[-1])[0:10]

    check = yf.download('^KS11', start_date_str, end_date_str, progress=False)
    check['signal'] = np.where((check['Close'] - check['Open'])> 0 , int(1), int(0))    

    for i in check.index:
        df.loc[(df.index == i), 'signal'] = check.loc[i, 'signal']

    df = df.dropna()

    return df

In [50]:
def clean_doc (doc):
    
    doc = re.sub(r'[^\wㄱ-ㅎ가-힣]', ' ', doc)
    doc = re.sub('  ', ' ', doc)
    doc = re.sub('  ', ' ', doc)

    return doc

In [66]:
df = pd.read_excel('NewsResult_20220315-20220331.xlsx')

In [67]:
train_df = df_process(df)

In [68]:
train_df

,제목,signal
일자,,
2022-03-16,"신세계百, 강남점 센텀시티점에 국내 최초 '베이비 디올' 매장 오픈",1.0
2022-03-16,"이동근 ""중대재해법, 기업 부담 커 윤석열 정부, 개선해달라""",1.0
2022-03-16,'위기에도 혁신' K-배터리 한자리 반도체 신화 넘본다,1.0
2022-03-16,[솔선守法] 중대재해법 첫 사건,1.0
2022-03-16,"코스피, 기관 매수에 상승 마감 하루만에 '7만전자' 회복",1.0
...,...,...
2022-03-30,"Sh수협은행, 내달부터 연 2.25% 정기예금 특판",0.0
2022-03-30,빅데이터로 동물병원을 스마트하게...6조 반려동물 시장 공략,0.0
2022-03-30,"삼진제약, 휴레이포지티브와 디지털 헬스케어 분야 협력",0.0


In [69]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 16799 entries, 2022-03-16 to 2022-03-30
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   제목      16799 non-null  object 
 1   signal  16799 non-null  float64
dtypes: float64(1), object(1)
memory usage: 393.7+ KB


In [70]:
train_df['제목'].to_list()

["신세계百, 강남점 센텀시티점에 국내 최초 '베이비 디올' 매장 오픈",
 '이동근 "중대재해법, 기업 부담 커 윤석열 정부, 개선해달라"',
 "'위기에도 혁신' K-배터리 한자리  반도체 신화 넘본다",
 '[솔선守法] 중대재해법 첫 사건',
 "코스피, 기관 매수에 상승 마감 하루만에 '7만전자' 회복",
 '환율, 4일 만에 하락 전환했지만 미 중 러發 악재 반영 이제 시작',
 '형지 I&C, "매출액 1000억원 회복 시동 건다"',
 '또 ‘제재’ 주고받은 서방 러 바이든, 24일 나토 정상회의 참석한다',
 '한국공대, K-Digital Training 훈련생 모집',
 '계양전기, 지난해 영업익 25억 전년비 8.9% ↓',
 "넥슨 주식 또 사들인 '오일머니' 3대 주주로 올라서",
 '여전사, 금리인하요구권 운용실적 공개해야',
 '셀트리온 불확실성 걷히자 바이오주 봄 기지개',
 '[특징주] 유가 하락 리오프닝 기대감에 항공주 강세',
 "따라 사면 물린다 급등 뒤 급락 속출 '어지러운 증시'",
 '롯데온, 한국생산성본부와 업무협약 “중소상공인 상품 판로 확대”',
 '"우크라이나 피해 中企, 최대 10억원 긴급자금 드려요"',
 '"유가 140불→99불"...우리 집앞 주유소 기름값은 언제 내리나?',
 '유영상 SKT 대표 작년 보수 15억원 박정호 부회장은 38억',
 'KOTRA, 백신기업 지원 유관기관 통합 온라인 설명회 개최',
 '중국산을 국산으로 고춧가루 원산지 속여 판 일당 구속',
 '경남TP 경남대 발전협의회 열어',
 '"될성싶은 스타트업 조기선점" KB인베, 초기투자 전담조직 신설',
 '경기도, 분양전환 시세차익 노린 공공임대주택 불법 투기 15명 적발',
 "사라 vs 사지 마라 '尹 수혜주' 한전에 엇갈리는 증권가 전망",
 '약물설계 기업 보로노이, 코스닥 상장 철회',
 '"포켓몬빵 100개 팔아요 근데 씰은 다 뺐어요" 황당 판매글',
 '"아웃백 질 떨어졌다" 소문에 bhc "레시피 유지, 법적 대응

In [71]:
train_df['제목'] = [clean_doc(x) for x in train_df['제목'].to_list()]

In [64]:
train_df

,제목,signal
일자,,
2022-03-16,신세계百 강남점 센텀시티점에 국내 최초 베이비 디올 매장 오픈,1.0
2022-03-16,이동근 중대재해법 기업 부담 커 윤석열 정부 개선해달라,1.0
2022-03-16,위기에도 혁신 K 배터리 한자리 반도체 신화 넘본다,1.0
2022-03-16,솔선守法 중대재해법 첫 사건,1.0
2022-03-16,코스피 기관 매수에 상승 마감 하루만에 7만전자 회복,1.0
...,...,...
2022-03-30,Sh수협은행 내달부터 연 2 25 정기예금 특판,0.0
2022-03-30,빅데이터로 동물병원을 스마트하게 6조 반려동물 시장 공략,0.0
2022-03-30,삼진제약 휴레이포지티브와 디지털 헬스케어 분야 협력,0.0


In [72]:
'''불용어 정리'''
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [81]:
han = Hannanum()

# train data 토큰화
train = []
for sentence in tqdm(train_df['제목']):
    tokenized_sentence = han.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    train.append(stopwords_removed_sentence)

100%|██████████| 16799/16799 [00:16<00:00, 1026.37it/s]


In [86]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train)

In [87]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

In [12]:
okt = Hannanum()

In [19]:
from konlpy.tag import Mecab

'-d /usr/local/lib/mecab/dic/mecab-ko-dic'

In [20]:
mcb = Mecab()
text = "안녕하세요 제 이름은 이선우 입니다."


In [21]:
mcb.pos(text, flatten=False, join=True)

[['안녕/NNG', '하/XSV', '세요/EP+EF'],
 ['제/NP'],
 ['이름/NNG', '은/JX'],
 ['이선우/NNP'],
 ['입니다/VCP+EF', './SF']]

In [22]:
mcb.morphs(text)

['안녕', '하', '세요', '제', '이름', '은', '이선우', '입니다', '.']

In [15]:
ret

NameError: name 'ret' is not defined

In [ ]:
!pip install eunjeon

  Using cached eunjeon-0.4.0.tar.gz (34.7 MB)
  Created wheel for eunjeon: filename=eunjeon-0.4.0-cp38-cp38-macosx_11_0_arm64.whl size=34077890 sha256=59a34a966b11252002d060f27a87a6815908d3f7613ecbb6fece3252237a65cb
  Stored in directory: /Users/Sun/Library/Caches/pip/wheels/4b/1d/e9/bfa4c18245fc292aaa650fe6dd982a4feddfaab49f3b8f9f25
Successfully built eunjeon


In [ ]:
from eunjeon import MeCab

m = MeCab()

m.pos("안녕하세요 제 이름은 이선우 입니다.")

ImportError: cannot import name 'MeCab' from 'eunjeon' (/opt/homebrew/Caskroom/miniforge/base/envs/NLP/lib/python3.8/site-packages/eunjeon/__init__.py)